In [158]:
import pandas as pd

# 작업디렉토리 설정

In [159]:
%cd /data/dhk/physionet.org/files/mimiciv/2.2/hosp

/data/dhk/physionet.org/files/mimiciv/2.2/hosp


# d_icd_diagnoses파일

In [160]:
df_d_icd_diagnoses = pd.read_csv('d_icd_diagnoses.csv')
df_d_icd_diagnoses.head()

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


In [161]:
#악성종양 icd_code 
# D00-D09는 암전조상태
df_d_icd_diagnoses_prep = df_d_icd_diagnoses[df_d_icd_diagnoses['icd_code'].str.match(r'^C[0-9]{2}|^D0[0-9]')]
df_d_icd_diagnoses_prep.head()

,icd_code,icd_version,long_title
13533,C00,10,Malignant neoplasm of lip
13534,C000,10,Malignant neoplasm of external upper lip
13535,C001,10,Malignant neoplasm of external lower lip
13536,C002,10,"Malignant neoplasm of external lip, unspecified"
13537,C003,10,"Malignant neoplasm of upper lip, inner aspect"


# diagnoses_icd파일

In [162]:
df_diagnoses_icd = pd.read_csv("diagnoses_icd.csv")
df_diagnoses_icd.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


# 두 dataframe 병합

In [163]:
#암 가진 환자들만 뽑음
df_merge1 = pd.merge(df_diagnoses_icd, df_d_icd_diagnoses_prep, on=['icd_code'])

df_merge1.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title
0,10001401,21544441,1,C675,10,10,Malignant neoplasm of bladder neck
1,10388043,26385522,13,C675,10,10,Malignant neoplasm of bladder neck
2,11163326,20175602,8,C675,10,10,Malignant neoplasm of bladder neck
3,11282936,22922093,1,C675,10,10,Malignant neoplasm of bladder neck
4,11444659,22007733,34,C675,10,10,Malignant neoplasm of bladder neck


# 암 가진 환자들의 다른 기저질환

In [164]:
# #병합된 df에서 subject_icd 추출
# masked_subject_id = df_merge1['subject_id'].values


In [165]:
# under_condition =  df_diagnoses_icd[df_diagnoses_icd['subject_id'].isin(masked_subject_id)][['subject_id','icd_code']]
# under_condition

# patients파일

In [166]:
df_patients = pd.read_csv("patients.csv")
df_patients.head(3)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN


# merge1과 patients 병합

In [167]:
merge2 = pd.merge(df_merge1,df_patients, on=['subject_id'])
merge2.head()
# len(merge2)
#이코드가 암환자 정보

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10001401,21544441,1,C675,10,10,Malignant neoplasm of bladder neck,F,89,2131,2014 - 2016,NaN
1,10001401,26840593,5,C679,10,10,"Malignant neoplasm of bladder, unspecified",F,89,2131,2014 - 2016,NaN
2,10001401,27060146,9,C679,10,10,"Malignant neoplasm of bladder, unspecified",F,89,2131,2014 - 2016,NaN
3,10388043,26385522,13,C675,10,10,Malignant neoplasm of bladder neck,F,44,2140,2014 - 2016,2142-10-31
4,10388043,22871741,4,C786,10,10,Secondary malignant neoplasm of retroperitoneu...,F,44,2140,2014 - 2016,2142-10-31


# 환자나이 계산하기 위해 merge

In [168]:
df_admission = pd.read_csv("admissions.csv")
merge2_admission = pd.merge(merge2,df_admission, on=['subject_id','hadm_id'])
merge2_admission

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title,gender,anchor_age,anchor_year,...,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10001401,21544441,1,C675,10,10,Malignant neoplasm of bladder neck,F,89,2131,...,P872K3,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0
1,10001401,26840593,5,C679,10,10,"Malignant neoplasm of bladder, unspecified",F,89,2131,...,P21SC7,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2131-06-19 17:14:00,2131-06-20 00:26:00,0
2,10001401,27060146,9,C679,10,10,"Malignant neoplasm of bladder, unspecified",F,89,2131,...,P08YAB,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2131-09-30 16:47:00,2131-10-01 02:24:00,0
3,10388043,26385522,13,C675,10,10,Malignant neoplasm of bladder neck,F,44,2140,...,P62F1I,PHYSICIAN REFERRAL,DIED,Other,ENGLISH,SINGLE,WHITE,2142-10-20 13:43:00,2142-10-21 02:08:00,1
4,10388043,26385522,5,C786,10,10,Secondary malignant neoplasm of retroperitoneu...,F,44,2140,...,P62F1I,PHYSICIAN REFERRAL,DIED,Other,ENGLISH,SINGLE,WHITE,2142-10-20 13:43:00,2142-10-21 02:08:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38530,18160122,29464470,4,C8460,10,10,"Anaplastic large cell lymphoma, ALK-positive, ...",M,67,2173,...,P43HX7,PHYSICIAN REFERRAL,NaN,Medicaid,?,MARRIED,BLACK/CAPE VERDEAN,2177-08-25 08:57:00,2177-08-26 19:19:00,0
38531,19159075,22141379,1,C44799,10,10,Other specified malignant neoplasm of skin of ...,M,91,2173,...,P62A9I,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE - OTHER EUROPEAN,NaN,NaN,0
38532,19290501,28187167,1,C8468,10,10,"Anaplastic large cell lymphoma, ALK-positive, ...",F,53,2179,...,P45GUA,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,NaN,NaN,0
38533,19515530,26697718,23,C4440,10,10,Unspecified malignant neoplasm of skin of scal...,M,83,2201,...,P61QZY,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0


# 입원시점에서 환자나이 계산

In [169]:
# 출생 연도 계산
print(merge2_admission[['anchor_year','anchor_age']].head(5))
print("~~")

##adgmit_age가 이상한 경우
#  환자의 나이가 89세 이상인 경우, 데이터 보호를 위해 모든 연령이 89세로 고정되어 있다는 것입니다.
merge2_admission = merge2_admission[merge2_admission['anchor_age']<89]

# admittime 열을 datetime 형식으로 변환
merge2_admission['birth_year'] = merge2_admission['anchor_year'] - merge2_admission['anchor_age']

# 입원 연도 추출
admit_year = pd.to_datetime(merge2_admission['admittime']).dt.year

# 입원 시점의 나이 계산
merge2_admission['admit_age'] = admit_year - merge2_admission['birth_year']

#인덱스 초기화
merge2_admission.reset_index(drop=True, inplace=True)

#결과확인
print(merge2_admission[['subject_id', 'admittime', 'admit_age','birth_year','anchor_year','anchor_age']].head())

   anchor_year  anchor_age
0         2131          89
1         2131          89
2         2131          89
3         2140          44
4         2140          44
~~
   subject_id            admittime  admit_age  birth_year  anchor_year  \
0    10388043  2142-10-21 00:53:00         46        2096         2140   
1    10388043  2142-10-21 00:53:00         46        2096         2140   
2    10388043  2142-10-21 00:53:00         46        2096         2140   
3    10388043  2142-10-21 00:53:00         46        2096         2140   
4    10388043  2142-10-21 00:53:00         46        2096         2140   

   anchor_age  
0          44  
1          44  
2          44  
3          44  
4          44  


/tmp/ipykernel_35788/4117411835.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge2_admission['birth_year'] = merge2_admission['anchor_year'] - merge2_admission['anchor_age']
/tmp/ipykernel_35788/4117411835.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge2_admission['admit_age'] = admit_year - merge2_admission['birth_year']


In [170]:
# merge2_admission.columns

# death비율과 death하지 않은비율

In [171]:
# merge2_admission['deathtime']
# NaN 값의 비율 계산
nan_ratio = merge2_admission['deathtime'].isna().mean()

# NaN이 아닌 값의 비율 계산
not_nan_ratio = 1 - nan_ratio

print(f"NaN 비율: {nan_ratio * 100:.2f}%")
print(f"NaN이 아닌 비율: {not_nan_ratio * 100:.2f}%")

NaN 비율: 94.52%
NaN이 아닌 비율: 5.48%


# death 종양환자 평균나이 비교

In [180]:
print(f"종양환자의 평균나이 :{ merge2_admission['admit_age'].mean() }")

C_death = merge2_admission[merge2_admission['deathtime'].notna()]
C_death_mean_age = C_death['admit_age'].mean()
print(f"death한 종양환자의 평균나이 :{C_death_mean_age}")

C_not_death = merge2_admission[merge2_admission['deathtime'].isna()]
C_not_death_mean_age = C_not_death['admit_age'].mean()
print(f"death하지않은 종양환자의 평균나이 :{C_not_death_mean_age}")




종양환자의 평균나이 :64.9079822616408
death한 종양환자의 평균나이 :65.72687861271676
death하지않은 종양환자의 평균나이 :64.86050603217159
